In [23]:
# Importaciones de bibliotecas necesarias
import pandas as pd  # Pandas para manipulación de datos tabulares
import json  # Módulo para trabajar con JSON
import ast  # Módulo para evaluar expresiones literales de Python
import re  # Módulo para trabajar con expresiones regulares
from textblob import TextBlob # Importa la clase TextBlob desde la biblioteca TextBlob
import nltk # Importa la biblioteca nltk (Natural Language Toolkit)
import csv # Importa el módulo csv en Python
import utils

# Habilita la recarga automática de módulos antes de ejecutar una celda
%load_ext autoreload
%autoreload 2

# Importa el módulo de advertencias y configura para ignorar todas las advertencias
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
'''  El objetivo del modelo de recomendación es generar una lista de 5 juegos 
al proporcionar el nombre de un juego o el ID de un usuario. 
Esta recomendación se basa en un puntaje que debe ser determinado. 
Para lograr esto, se examinan los dataframes df_reviews y df_items, los cuales fueron analizados previamente.'''

'  El objetivo del modelo de recomendación es generar una lista de 5 juegos \nal proporcionar el nombre de un juego o el ID de un usuario. \nEsta recomendación se basa en un puntaje que debe ser determinado. \nPara lograr esto, se examinan los dataframes df_reviews y df_items, los cuales fueron analizados previamente.'

In [ ]:
''' En esteNotebook se realiza el Análisis Exploratorio de los datos de los tres conjuntos de datos de lla consigna
ya limpios para elegir  las variables que serán usadas para entrenar el modelo de recomendación solicitado.
Esto reducirá el tamaño de los df y solo incluirá los datos necesarios'''

In [ ]:
# Abro archivo user_reviews y lo renombro df_reviews para diferenciarlo de los APIs

ruta_df_user_reviews = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_user_reviews.parquet"
df_reviews = pd.read_parquet(ruta_df_user_reviews)
df_reviews.head()

In [ ]:
# Hago merge con la columna helpful del dataframe original del 
# user_reviews. para eso abro el dataframe df_user_reviews_sentimiento. para eso abro el df que hice en eda_users_reviews_helpful
# Hago esta vuenta por que el df_reviews ya está limpio

ruta_df_user_reviews_helpful = r"C:\Users\Usuario\Henry\PI1_ML\Modelo_Aprendizaje_Automático\EDA_Datos_Modelo\df_user_reviews_helpul.parquet"
df_user_reviews_helpul = pd.read_parquet(ruta_df_user_reviews_helpful)
df_user_reviews_helpul.head()


In [26]:
# Hago Merge

df_reviews = pd.merge(df_reviews, df_user_reviews_helpul, on='item_id')


In [ ]:
df_reviews.head()

In [28]:
df_reviews.shape

(43706993, 7)

In [29]:
# Borro los NA

df_reviews = df_reviews.dropna()

In [30]:
# retiro duplicados
df_reviews = df_reviews.drop_duplicates()

In [31]:
df_reviews.shape

(1741916, 7)

In [ ]:
# Renombro columnas para saber que son originales y no modificadas

df_reviews = df_reviews.rename(columns={'posted': 'reviews_date'})
df_reviews = df_reviews.rename(columns={'item_id': 'reviews_item_id'})
df_reviews = df_reviews.rename(columns={'recommend': 'reviews_recommend'})
df_reviews = df_reviews.rename(columns={'helpful': 'reviews_helpful'})

df_reviews.head()


In [ ]:
df_reviews.info()

In [34]:
df_reviews.to_parquet('df_reviews.parquet')

In [35]:
# Reviso número de registros

print(f'La cantidad de registros que hay son: {df_reviews.shape[0]}')

La cantidad de registros que hay son: 1741916


In [36]:
# Pero realmente necesito solo los registros que tengan información en reviews_helpful     
# así que borro los que tengan en esa casillas y para eso voy a revisar los valores unicos

In [ ]:
valores_unicos = df_reviews['reviews_helpful'].unique()
valores_unicos
print(valores_unicos)

In [ ]:
df_reviews.info()

In [39]:
df_reviews.shape

(1741916, 7)

In [40]:
# Voy a borrar la filas que contengan 'No ratings yet'

filas_a_borrar = df_reviews[df_reviews['reviews_helpful'] == 'No ratings yet'].index
df_reviews = df_reviews.drop(filas_a_borrar)

In [41]:
# Ahora si cuento cuantos raiting tengo 

df_reviews.shape

(1685956, 7)

In [42]:
# Numero de raitings para el modelo

print(f'La cantidad de registros que hay son: {df_reviews.shape[0]}')

La cantidad de registros que hay son: 1685956


In [ ]:
# Ahora no tengo como contar y revisar porcentajes toca hacer una función resumen_cant_porcentaje para para resumir la cantidad y el porcentaje de 
#ocurrencias de cada valor en una columna específica.

# En el archivo Datos_Modelo voy a incluir la función y luego la llamo en este notbook. voy a utilizar convencion utils s ver si pueo mantener el codigo organizado

# Me tocó hacer un libro en py para utilizar la función que haga

In [43]:
# Exploro columna reviws_recommend
utils.resumen_cant_porcentaje(df_reviews, 'reviews_recommend')

,Cantidad,Porcentaje
reviews_recommend,,
True,1502685,89.13
False,183271,10.87


In [44]:
# Exploro columna sentimenr_analysis

utils.resumen_cant_porcentaje(df_reviews, 'sentiment_analysis')

,Cantidad,Porcentaje
sentiment_analysis,,
2,936839,55.57
1,402788,23.89
0,346329,20.54


In [ ]:
''' de estos resultados puedo concluir qe en cuento a comentarios positivos  o sea reviews recommend el 89% son comentarios positivos
y en cuento al sentyment analisis  el 23% es neutral, el 20,54% tiene un punto de vista negativo pero el 93,6 considera un item positivo

In [45]:
# Ahora quiero  ver la cantidad de recomendaciones para los distintos sentimientos de las reviews. asi que voy a agrupar por 'sentiment_analysis' y 'reviews_recommend'

df_agrupado = df_reviews.groupby(['sentiment_analysis', 'reviews_recommend'])


In [46]:
# Se cuentan la cantidad de reviews en cada grupo
cantidad_grupo = df_agrupado.size()

In [47]:
# Se calcula el porcentaje por cada grupo
porcentaje_grupo = 100 * df_agrupado.size() / len(df_reviews)

In [48]:
sentimiento_recomendacion = pd.DataFrame({
    "Cantidad": cantidad_grupo,
    "Porcentaje": porcentaje_grupo
})

In [49]:
sentimiento_recomendacion

Cantidad  Porcentaje
sentiment_analysis reviews_recommend                      
0                  False                 86484    5.129671
                   True                 259845   15.412324
1                  False                 42434    2.516910
                   True                 360354   21.373867
2                  False                 54353    3.223868
                   True                 882486   52.343359

In [ ]:
# Se puede ver que el porcentaje respecto del total de los reviews que casi el 54% de los reviews que reflejan un sentimiento neutral
# recomienda el juego por lo que un sentimiento neutral podría estar relacionado con un comentario sencillo sin muchos detalles pero 
# que aún así el juego le gustó como para recomendar. Por otra parte, de los comentarios que reflejaron un sentimiento negativo cerca 
# del 6% aún recomienda el juego, lo que es algo contradictorio.